# L11b: The POP of Composite Contracts and A New Role for Delta
In this lecture, we will explore the probability of profit (POP) and the probability of exercise (POE) for composite options contracts. We will also introduce a new concept called hedging, which uses our old friend Delta in a new way.

> __Learning Objectives:__
>
> By the end of this lecture, you should be able to:
> * __Calculate probability of profit and exercise for composite contracts:__ Compute POP by evaluating whether the total profit from all legs is positive, and determine POE separately for each individual leg based on whether that option finishes in-the-money at expiration.
> * __Analyze straddle positions using breakeven analysis:__ Construct profit diagrams for long and short straddles, identify the two breakeven points where the position transitions from profit to loss, and calculate POP using projected share price distributions.
> * __Construct delta-neutral positions using composite contracts:__ Calculate the net Delta for a portfolio containing options and underlying shares, and determine the number of shares needed to hedge the options position and achieve a delta-neutral portfolio.

Let's get started!
___

## Examples
Today, we will use the following examples to illustrate key concepts:

> [▶ Compute the POP and POE for a straddle on AMD](CHEME-5660-L11b-Example-Straddle-AMD-Earnings-Fall-2025.ipynb). In this example, we'll compute the probability of profit (POP) and the probability of exercise (POE) for a long straddle composite options contract on `AMD` using projected share price distributions and compare with the delta ($\Delta$) estimate of probability of exercise for each leg of the straddle.

> [▶ Compute the Delta for a vertical spread on AMD](CHEME-5660-L11b-Example-DeltaVerticalSpread-Fall-2025.ipynb). In this example, we'll compute the Delta for a vertical spread composite options contract on `AMD` using the CRR binomial lattice model and demonstrate how Delta can be used to construct a delta-neutral hedging position.
___

## Concept Review: Options, The Greeks and POP/POE calculations
Let's start with a quick review of options contracts, the Greeks, and how to calculate POP and POE for simple options contracts.

<div>
    <center>
        <img src="figs/Fig-Options-Summary-Schematic.png" width="800"/>
    </center>
</div>

The Greeks are a set of (local) measures that describe how the price of an option changes in response to changes in various parameters. 

> __The Greeks__ 
> 
> - **Delta ($\Delta$)**: Measures the sensitivity of the option's price to a `+1 USD/share` change in the underlying asset's price. However, $\Delta$ also represents the hedge ratio, i.e., the number of shares of the underlying asset needed to hedge one option contract, and the probability that the option will finish in-the-money at expiration.
> - **Theta ($\Theta$)**: Measures the sensitivity of the option's price to a `1-day decrease` in the lifetime of the contract (time decay). Theta benefits option sellers, as options lose value as expiration approaches, all else being equal.
> - **Vega ($\nu$)**: Measures the sensitivity of the option's price to a `+1%` change in the volatility of the underlying asset. The higher the volatility, the higher the option premium, as there is a greater chance of significant price movements.
> - **Gamma ($\Gamma$)**: Measures the sensitivity of $\Delta$ to a `+1 USD/share` change in the underlying asset's price. This is the second derivative of the option price with respect to the underlying asset price.
> - **Rho ($\rho$)**: Measures the sensitivity of the option's price to a `+1%` change in the risk-free interest rate. This is generally a minor effect for most options, but can be more significant for longer-term options.
>
> When trading, most trading platforms will compute these for you automatically!

We've already seen an example of the hedge ratio interpretation of $\Delta$ in the last example notebook. Let's look at the probability interpretation of $\Delta$. In particular, we can use $\Delta$ to estimate the probability of exercise: 

> __Probability of Exercise (POE):__
>
> The probability of exercise (POE) is the probability that at the expiration of the (composite or single) contract at time $t = T$, the option will finish in-the-money, i.e.,
> $$
\boxed{
\begin{align*}
\text{POE} & = \begin{cases}
\mathbb{P}\left(S_{T} > K + 0.01\right) & \text{(for Call Options)} \\
\mathbb{P}\left(S_{T} < K - 0.01\right) & \text{(for Put Options)}
\end{cases}
\end{align*}}
$$
> where $S_{T}$ is the share price of the underlying asset at expiration and $K$ is the strike price. Delta provides an approximation of the probability of exercise.

We can also compute the __Probability of Profit (POP)__ for an option contract, which is closely related to the probability of exercise.

> __Probability of Profit (POP):__ 
>
> The probability of profit (POP) is the probability that at the expiration of the (composite or single) contract at time $t = T$, the profit of the contract is positive, i.e.,
> $$
\boxed{
\begin{align*}
\text{POP} & = \mathbb{P}\left(\hat{P}_{\mathcal{C}} > 0\right)
\end{align*}}
$$
> where $\hat{P}_{\mathcal{C}}$ is the profit of the (single or composite) contract at expiration. 
___

## What is POP/POE of Composite Contracts?
We've already discussed how to calculate the POP and POE for simple options contracts. Now, let's extend that understanding to composite contracts, such as spreads and straddles. Let's start with a quick review of what a composite contract is.

### Composite Options Contracts (originally from L10a)
Let $\mathcal{C}$ be a composite contract with $d$ legs (individual short/long put/call contracts) where each leg is written with respect to the same underlying asset XYZ and same expiration date. 
The payoff of the composite contract $\hat{V}_{\mathcal{C}}(S(T),K_{1},\dots,K_{d})$ is given by:
$$
\begin{align*}
\hat{V}_{\mathcal{C}}(S(T),K_{1},\dots,K_{d}) &= \sum_{i\in\mathcal{C}}\theta_{i}\;{n_{i}}\;\underbrace{V_{i}^{(\tau_{i})}(S(T),K_{i})}_{\text{payoff of contract } i}
\end{align*}
$$
where $K_{i}$ denotes the strike price of contract $i$, $V_{i}^{(\tau_{i})}(S(T),K_{i})$ is the payoff of contract $i$ (with superscript $\tau_{i} \in \{c, p\}$ indicating call or put), and the term $\theta_{i}$ is the contract orientation (sense) for contract $i$: 
$$
\theta_{i}=\begin{cases}-1 & \text{if contract } i \text{ is short (sold),}\\
1 & \text{if contract } i \text{ is long (bought),}\end{cases}
$$
and $n_{i}$ denotes the copy number of contract $i$. The profit of the composite contract $\hat{P}$ at expiration is given by:
$$
\begin{align*}
\hat{P}_{\mathcal{C}}(S(T),K_{1},\dots,K_{d}) &= \sum_{i\in\mathcal{C}}\theta_{i}\;{n}_{i}\;\underbrace{P_{i}^{(\tau_{i})}(S(T),K_{i})}_{\text{profit of contract } i}\\
& = \sum_{i\in\mathcal{C}}\theta_{i}\;{n}_{i}\;\Bigl(V_{i}^{(\tau_{i})}(S(T),K_{i}) -  \mathcal{P}_{i}^{(\tau_{i})}(K_{i},S(0))\Bigr)
\end{align*}
$$
where $P_{i}^{(\tau_{i})}(S(T),K_{i})$ is the profit of contract $i$, and $\mathcal{P}_{i}^{(\tau_{i})}(K_{i},S(0))$ represents the premium for contract $i$, with the superscript $\tau_{i} \in \{c, p\}$ indicating whether contract $i$ is a call option ($\tau_{i} = c$) or put option ($\tau_{i} = p$).

Thus, the payoff and profit of a composite contract are simply the sum of the payoffs and profits of the individual legs, weighted by their orientation and copy number.

### POP/POE for Composite Contracts
The probability of profit (POP) for a composite contract $\mathcal{C}$ is easily computed using the profit function defined above:
$$
\begin{align*}
\text{POP}_{\mathcal{C}} &= \mathbb{P}\Big(\hat{P}_{\mathcal{C}}(S(T),K_{1},\dots,K_{d}) > 0\Big)\\
&= \mathbb{P}\Big(\sum_{i\in\mathcal{C}}\theta_{i}\;{n}_{i}\;\Bigl(V_{i}^{(\tau_{i})}(S(T),K_{i}) -  \mathcal{P}_{i}^{(\tau_{i})}(K_{i},S(0))\Bigr) > 0\Big)
\end{align*}
$$
However, calculating the probability of exercise (POE) for composite contracts is more nuanced. Unlike single contracts where POE has a clear interpretation, POE for composite contracts is strategy-specific and must be computed for each leg individually. For a composite contract $\mathcal{C}$, we compute the POE of each leg $i$ separately:
$$
\text{POE}_{i} = \begin{cases}
\mathbb{P}(S(T) > K_{i}) & \text{if leg } i \text{ is a call option} \\
\mathbb{P}(S(T) < K_{i}) & \text{if leg } i \text{ is a put option}
\end{cases}
$$
The POE values for individual legs provide insight into the likelihood of exercise or assignment for each position in the composite contract.

In both cases, we use a mathematical model of the future share price distribution (e.g., generated using Geometric Brownian Motion) to compute these probabilities. 

### Analysis of a Straddle Position
We now apply these concepts to a straddle, a composite contract with two legs. A [straddle](https://www.investopedia.com/terms/s/straddle.asp) is a _neutral strategy_ constructed by simultaneously buying (or selling) a put and a call option on the same underlying asset `XYZ`, with the _same expiration_, and the _same strike price_. 

> __Long (short) straddle__
> 
> For a long straddle, we purchase (are long, $\theta = 1$) put and call contracts on the same underlying asset, where each contract has the same expiration date. For a short straddle, we sell (are short, $\theta = -1$) put and call contracts on the same underlying asset, where each contract has the same expiration date. For a straddle, both legs have the same strike price $K$.


#### Profit and Breakeven Analysis for a Straddle
Let $K_{j}$ denote the strike price of contract $j$ (USD/share), where the premium of contract $j$ is $\mathcal{P}_{j}$ (USD/share). Finally, let index $j=1$ denote the put contract, $j=2$ denote the call contract; for a straddle $K_{1}= K_{2}\equiv{K}$ (both legs have the same strike). The maximum profit (short) or loss (long) for a single straddle composite contract $\hat{P}_{\mathcal{C}}$ at expiration is given by:

$$\hat{P}_{\mathcal{C}} = \theta\cdot\left(P_{1}^{(p)}+P_{2}^{(c)}\right)$$

where $\theta_{1}=\theta_{2}\equiv\theta$ denotes a direction parameter: $\theta=-1$ if each leg is sold (short), $\theta=1$ otherwise. After substitution of the profit functions for a put and a call contract, the overall profit for a straddle $\hat{P}_{\mathcal{C}}$ is given by:

$$\hat{P}_{\mathcal{C}} = \theta\cdot\Bigl[(K-S(T))^{+}+(S(T)-K)^{+}-(\mathcal{P}_{1}+\mathcal{P}_{2})\Bigr]$$

where $V_{p}(K,S(T)) = (K-S(T))^{+}=\max(K-S(T),0)$ is the payoff function for the put contract, and $V_{c}(K,S(T)) = (S(T)-K)^{+} = \max(S(T)-K,0)$ is the payoff function for the call contract. The profit (or loss) of a straddle has three regimes given by:

$$
\hat{P}_{\mathcal{C}} = \begin{cases}
  \theta\cdot\Bigl[(S(T)-K)-\left(\mathcal{P}_{1}+\mathcal{P}_{2}\right)\Bigr]  & S(T)>K \\
  -\theta\cdot\Bigl[\mathcal{P}_{1}+\mathcal{P}_{2}\Bigr] & S(T)=K \\
    \theta\cdot\Bigl[(K-S(T))-\left(\mathcal{P}_{1}+\mathcal{P}_{2}\right)\Bigr] & S(T)<K
\end{cases}
$$

Finally, a straddle has _two_ possible breakeven points denoted as $S^{+}$ and $S^{-}$:
* If $S(T)>K$: a straddle will break even at $S^{+} = K + \left(\mathcal{P}_{1}+\mathcal{P}_{2}\right)$
* If $S(T)<K$: a straddle will break even at $S^{-} = K - \left(\mathcal{P}_{1}+\mathcal{P}_{2}\right)$.
   
Let's compute the POP and POE for a straddle position in AMD.

> __Example__
>
> [▶ Compute the POP and POE for a straddle on AMD](CHEME-5660-L11b-Example-Straddle-AMD-Earnings-Fall-2025.ipynb). In this example, we'll compute the probability of profit (POP) and the probability of exercise (POE) for a long straddle composite options contract on `AMD` using projected share price distributions and compare with the delta ($\Delta$) estimate of probability of exercise for each leg of the straddle.

___

## Delta as a Hedging Tool for Composite Contracts
In addition to its interpretations as a sensitivity measure and a probability estimate, Delta ($\Delta$) also plays an important role in hedging strategies for options contracts, including composite contracts.

> __What is Hedging?__ 
>
> Hedging is a risk management strategy used to offset potential losses in an investment by taking an opposite position in a related __counter asset__. In the context of options trading, hedging often involves using the underlying asset to mitigate the risk associated with holding options contracts, or buying/selling other options contracts to offset potential losses.

The goal of a perfectly (price) hedged position is to be insensitive to small changes in the price of the underlying asset. This is achieved by constructing a composite position (portfolio of options, options and the underlying asset, etc.) where the overall Delta ($\Delta$) is zero.

### What is the Delta of a Composite Contract?
Suppose a composite contract $\mathcal{C}$ with legs $i=1,\dots,|\mathcal{C}|$ is constructed using American-style options contracts on the same underlying firm (share price $S$) with the same expiration date $T$. 
If you hold $\theta_{i}n_i$ contracts of leg $i$ ($\theta = 1$ for long, and $\theta = -1$ for short) and the contract multiplier is $m$ shares/contract, and you also hold $N_S$ shares of the stock, then your portfolio value at time $t$ is given by:
$$
\Pi(S,t)=N_S S+m\sum_{i=1}^{|\mathcal{C}|} \theta_i n_i \mathcal{P}_i(S,t),
$$
The (spot) Delta is the sensitivity of the value of the portfolio to movements in the underlying asset price $S$:
$$
\begin{align*}
\Delta_{\text{net}}(S,t)&=\frac{\partial \Pi}{\partial S} \\
&= \underbrace{\frac{\partial (N_S S)}{\partial S}}_{=N_S} + m\sum_{i=1}^{|\mathcal{C}|} \theta_i n_i \underbrace{\frac{\partial \mathcal{P}_i}{\partial S}}_{=\Delta_i(S,t)}\\
&= N_S + \underbrace{m\sum_{i=1}^{|\mathcal{C}|} \theta_i n_i \Delta_i(S,t)}_{= \Delta_{\text{opts}}(S,t)}
\end{align*}
$$
The $N_S$ term is the stock's contribution to the net portfolio Delta. If you hold **no** stock, set $N_S=0$ and it disappears. A perfectly delta-hedged position has $\Delta_{\text{net}}(S,t)=0$ for all $t\in[0,T]$ and $S>0$. 

> __Delta-neutral position:__ A position is delta-neutral if $\Delta_{\text{net}}(S,t)=0$ for all $t\in[0,T]$ and $S>0$. This can be achieved by adjusting the stock position $N_S$ to offset the options position's Delta $\Delta_{\text{opts}}(S,t)$:
> $$
\begin{align*}
\Delta_{\text{net}}(S,t)&= N_S + \Delta_{\text{opts}}(S,t) = 0 \\
\Rightarrow N_S & = - \Delta_{\text{opts}}(S,t)
\end{align*}
$$

Let's do a really quick gut-check calculation to see how this works in practice.

> __Quick gut-check example:__ Short $10$ calls at $\Delta = 0.6$ with $m=100$ shares/contract $\Rightarrow$ $\Delta_{\text{opts}}=10\times(-1)\times 0.6\times 100=-600$. If you also hold $N_S=+600$ shares, then $\Delta_{\text{net}}=600-600=0$ (delta-neutral).

Finally, if a leg $i$ is exercised/assigned at some $\tau\le T$, set that leg's $\Delta_i(\cdot)=0$ for $t\ge \tau$ and update the stock position:
* Call leg $i$: $N_S \leftarrow N_S + \theta_i \cdot m \cdot n_i$.
* Put leg $i$: $N_S \leftarrow N_S - \theta_i \cdot m \cdot n_i$.



Let's look at an example of Delta of a composite contract by analyzing a vertical (credit) spread composed of two call (or put) options.

> __Example__
>
> [▶ Compute the Delta for a vertical spread on AMD](CHEME-5660-L11b-Example-Delta-VerticalSpread-Fall-2025.ipynb). In this example, we'll compute the Delta for a vertical spread composite options contract on `AMD` using the CRR binomial lattice model and demonstrate how Delta can be used to construct a delta-neutral hedging position.

___

## Summary
This lecture extends probability of profit and probability of exercise concepts to composite options contracts and introduces Delta as a hedging tool.

> __Key Takeaways:__
> 
> * __Composite contract POP and POE extend single contract measures:__ POP for composite contracts is the probability that the sum of all leg profits is positive. POE is computed separately for each leg based on whether that individual option finishes in-the-money. Both probabilities use projected share price distributions at expiration.
> * __Straddles demonstrate composite contract analysis:__ Composite contracts combine multiple option legs to create positions with different profit characteristics than single contracts. The straddle illustrates how to compute breakeven points by summing premiums across legs, construct profit diagrams by adding individual leg payoffs, and calculate probability of profit for positions with multiple expiration scenarios.
> * __Delta enables construction of hedged positions:__ The net Delta of a portfolio equals the stock position plus the sum of weighted option Deltas. A delta-neutral position has net Delta equal to zero, achieved by holding shares equal to the negative of the options position Delta to offset the options exposure.

Understanding composite contract probabilities and Delta hedging enables construction and management of complex options strategies.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___